## Fitting with our dataset

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import gepard as g
import gepard.plots as gplots
from gepard.fits import GLO15new, AUTIpts, ALUIpts
print('Gepard version = {}'.format(g.__version__))

Gepard version = 0.9.11b0


In [3]:
#import gepard as g
#print(g.__file__)

import torch
import numpy as np 
#np.set_printoptions(legacy='1.25')
import matplotlib
import matplotlib.pyplot as plt
import shelve, logging, copy
logging.basicConfig(level=logging.ERROR) 
import pandas as pd 
from scipy.integrate import quad
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.stats import norm


In [4]:
# To have nice LaTeX fonts on plots
plt.rc('text', usetex=True)
params = {'text.latex.preamble' : '\n'.join([r'\usepackage{amssymb}', r'\usepackage{amsmath}'])}
plt.rcParams.update(params)

In [20]:
g.describe_data(g.dset[94])

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
10 x ALU     CLAS    -1     94  arXiv:1501.07052
----------------------------------------------
TOTAL = 10


In [5]:
fitpoints =  g.dset[98] + g.dset[91] #g.dset[102] it worked #g.dset[7] it worked # g.dset[6] didn't work # + g.dset[8] it did work # + g.dset[82] # mydset[325] + mydset[98], dset[91] did not work
g.describe_data(fitpoints)

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
166 x ALU     CLAS    N/A    91  arXiv:1501.07052
2640 x XUU     CLAS    N/A    98  arXiv:1504.02009
----------------------------------------------
TOTAL = 2806


In [22]:
fitpoints =  g.dset[94]    # To make life simple
g.describe_data(fitpoints)

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
10 x ALU     CLAS    -1     94  arXiv:1501.07052
----------------------------------------------
TOTAL = 10


In [6]:
# For easier manipulations we transform some datasets to pandas frames
#data = mydset[325]
#BSA = data.df()  # Beam Spin Asymmetry CLAS 2022

data = g.dset[91] #[8] #[82]
BSA = data.df() 

data = g.dset[98]
BSS = data.df()  # Cross Section CLAS 2015 

### Fit with NN  - Q2 dependence of CFFs

In [13]:
class NN(g.model.NeuralModel, g.eff.KellyEFF, g.dvcs.BM10tw2):

    def build_net(self):
            '''Overriding the default architecture and optimizer'''
            nn_model = torch.nn.Sequential(
                    torch.nn.Linear(3, 23),
                    torch.nn.ReLU(),
                    torch.nn.Linear(23, 37),
                    torch.nn.ReLU(),
                    torch.nn.Linear(37, 19),
                    torch.nn.ReLU(),
                    torch.nn.Linear(19, len(self.output_layer))
                )
            optimizer = torch.optim.Rprop(nn_model.parameters(), lr=0.01)
            return nn_model, optimizer

In [23]:
g.describe_data(fitpoints)
th = NN(output_layer=['ImH', 'ImE', 'ImHt'], q2in=True)#'ImH', 'ReH', 'ImE', 'ReE', 'ImHt', 'ReHt', 'ImEt', 'ReEt'])#
print(th.useDR)

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
10 x ALU     CLAS    -1     94  arXiv:1501.07052
----------------------------------------------
TOTAL = 10
None


In [24]:
f = g.fitter.NeuralFitter(fitpoints, th, nnets=1, batchlen=10, regularization='L2', lx_lambda=0.002)
#f.fitgood()
f.fit()


Epoch  10: train error = 3.2456 test error = 5.1101 -
Epoch  20: train error = 1.0444 test error = 5.5378 +
Epoch  30: train error = 0.4389 test error = 7.6208 +
Epoch  40: train error = 0.2747 test error = 5.3100 +
Epoch  50: train error = 0.0938 test error = 4.7405 -
Epoch  60: train error = 0.0705 test error = 5.0928 +
Epoch  70: train error = 0.0662 test error = 5.0932 +
Epoch  80: train error = 0.0628 test error = 5.1825 +
Epoch  90: train error = 0.0604 test error = 5.2428 +
Epoch 100: train error = 0.0589 test error = 5.2353 +
No improvement for 5 batches. Stopping early.
Net 0 --> test_err = 4.740548376365378


In [25]:
th.chisq(fitpoints)

(23.832529921944662, 10, 0.008057532676180701)

### Fit with NN - no Q2 dependence

In [26]:
class NN(g.model.NeuralModel, g.eff.KellyEFF, g.dvcs.BM10tw2):

    def build_net(self):
            '''Overriding the default architecture and optimizer'''
            nn_model = torch.nn.Sequential(
                    torch.nn.Linear(2, 23),
                    torch.nn.ReLU(),
                    torch.nn.Linear(23, 37),
                    torch.nn.ReLU(),
                    torch.nn.Linear(37, 19),
                    torch.nn.ReLU(),
                    torch.nn.Linear(19, len(self.output_layer))
                )
            optimizer = torch.optim.Rprop(nn_model.parameters(), lr=0.01)
            return nn_model, optimizer

In [27]:
g.describe_data(fitpoints)
th = NN(output_layer=['ImH', 'ImE', 'ImHt'], q2in=False)#'ImH', 'ReH', 'ImE', 'ReE', 'ImHt', 'ReHt', 'ImEt', 'ReEt'])#
print(th.useDR)

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
10 x ALU     CLAS    -1     94  arXiv:1501.07052
----------------------------------------------
TOTAL = 10
None


In [28]:
f = g.fitter.NeuralFitter(fitpoints, th, nnets=1, batchlen=10, regularization='L2', lx_lambda=0.002)
#f.fitgood()
f.fit()


Epoch  10: train error = 2.6676 test error = 4.3236 -
Epoch  20: train error = 0.9335 test error = 5.6773 +
Epoch  30: train error = 0.8481 test error = 9.8181 +
Epoch  40: train error = 0.7032 test error = 25.9396 +
Epoch  50: train error = 0.6050 test error = 28.5054 +
Epoch  60: train error = 0.4879 test error = 35.8781 +
No improvement for 5 batches. Stopping early.
Net 0 --> test_err = 4.3236270375964105


In [29]:
th.chisq(fitpoints)

(113.9823641380506, 10, 0.0)

### Fit $\phi$-dependent data with NN  - Q2 dependence of CFFs

In [30]:
fitpoints =  g.dset[91]
g.describe_data(fitpoints)

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
166 x ALU     CLAS    N/A    91  arXiv:1501.07052
----------------------------------------------
TOTAL = 166


In [31]:
class NN(g.model.NeuralModel, g.eff.KellyEFF, g.dvcs.BM10tw2):

    def build_net(self):
            '''Overriding the default architecture and optimizer'''
            nn_model = torch.nn.Sequential(
                    torch.nn.Linear(3, 23),
                    torch.nn.ReLU(),
                    torch.nn.Linear(23, 37),
                    torch.nn.ReLU(),
                    torch.nn.Linear(37, 19),
                    torch.nn.ReLU(),
                    torch.nn.Linear(19, len(self.output_layer))
                )
            optimizer = torch.optim.Rprop(nn_model.parameters(), lr=0.01)
            return nn_model, optimizer

In [32]:
g.describe_data(fitpoints)
th = NN(output_layer=['ImH', 'ImE', 'ImHt'], q2in=True)#'ImH', 'ReH', 'ImE', 'ReE', 'ImHt', 'ReHt', 'ImEt', 'ReEt'])#
print(th.useDR)

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
166 x ALU     CLAS    N/A    91  arXiv:1501.07052
----------------------------------------------
TOTAL = 166
None


In [33]:
f = g.fitter.NeuralFitter(fitpoints, th, nnets=1, batchlen=10, regularization='L2', lx_lambda=0.002)
#f.fitgood()
f.fit()


Epoch  10: train error = 2.3330 test error = 2.7404 -
Epoch  20: train error = 2.1260 test error = 2.8656 +
Epoch  30: train error = 2.0212 test error = 3.3957 +
Epoch  40: train error = 1.9349 test error = 4.2639 +
Epoch  50: train error = 1.8188 test error = 5.2953 +
Epoch  60: train error = 1.7972 test error = 5.5116 +
No improvement for 5 batches. Stopping early.
Net 0 --> test_err = 2.740393877029419


In [34]:
th.chisq(fitpoints)

(297.5345699060113, 166, 1.5858557800285666e-09)

In [35]:
th.chisq(g.dset[94])

(9.853285455460773, 10, 0.45345809908245815)

(Interestingly, this rather slow fit to phi-dependent data describes harmonics better than the fit to harmonics.)

### Use shelve to store the trained models

In [14]:
# create a shelf file 
shelve_file = shelve.open("Models") 

# Store num list in shelf file 
shelve_file['NoDR'] = th.nets
  
# now, we simply close the shelf file. 
shelve_file.close()

In [14]:
shelve_file = shelve.open("Models") 
  
NoDR = shelve_file['NoDR']

th4 = NNTest(output_layer=['ImH', 'ReH'])
th4.nets = NoDR

# print keys list 
#print(f"Keys = {list(shelve_file.keys())}") 
  
# now, we simply close the shelf file. 
shelve_file.close() 